<a href="https://colab.research.google.com/github/norightt/python-machine-learning/blob/main/boosting_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


В данных нет пропусков, поэтому ничего не придется заполнять во время кодирования

In [ ]:
df.isnull().values.any()

False

Признаком, из-за которого возможен лик в данных является salary, так как он в большинстве случаев просто дублирует таргет salary_in_usd и не может быть использован при построении моделей

In [ ]:
df = df.drop('salary', axis=1)

Кодируем, чтобы не получилось разное число признаков на выборках

In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = (df.dtypes == "object").values

df_real = df[df.columns[~categorical_features]]
df_cat = df[df.columns[categorical_features]]

df_ohe = pd.get_dummies(df_cat, drop_first = True, dtype = float)

df_encoded = pd.concat([df_real, df_ohe], axis=1)

Выделение таргета + сплит на выборки

In [ ]:
y = df_encoded['salary_in_usd']
df_encoded = df_encoded.drop('salary_in_usd', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_temp, y_train, y_temp = train_test_split(df_encoded, y, random_state = 42, test_size = 0.2)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, random_state = 42, test_size = 0.5)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np

lin_reg = LinearRegression().fit(x_train, y_train)

In [ ]:
print(f'MAPE: {mean_absolute_percentage_error(y_test, lin_reg.predict(x_test))}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, lin_reg.predict(x_test)))}')

MAPE: 342120328.396346
RMSE: 97932104534349.98


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
model_xgb = XGBRegressor()

params1 = {"max_depth": range(2, 7, 2), "min_child_weight": range(1, 6, 2)}

grid_search1 = GridSearchCV(estimator=model_xgb, param_grid=params1, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
grid_search1.fit(x_train, y_train)
print(grid_search1.best_params_, grid_search1.best_score_)

y_pred = grid_search1.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'max_depth': 2, 'min_child_weight': 3} -47787.757209674564
MAPE: 0.3429890902773069
RMSE: 50486.60133538425


In [ ]:
params2 = {"max_depth": [3, 4, 5], "min_child_weight": [2, 3, 4]}
grid_search2 = GridSearchCV(estimator=model_xgb, param_grid=params2, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
grid_search2.fit(x_train, y_train)
print(grid_search2.best_params_, grid_search2.best_score_)

y_pred = grid_search2.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'max_depth': 3, 'min_child_weight': 2} -47738.98669767151
MAPE: 0.3355965140985402
RMSE: 50065.091305473514


In [ ]:
model_xgb.max_depth = 3
model_xgb.min_child_weight = 2

In [ ]:
params3 = {"gamma": np.arange(0,0.5, 0.1), 'learning_rate' : np.arange(0.01, 0.3, 0.2)}
grid_search3 = GridSearchCV(estimator=model_xgb, param_grid=params3, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
grid_search3.fit(x_train, y_train)
print(grid_search3.best_params_, grid_search3.best_score_)

y_pred = grid_search3.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'gamma': 0.0, 'learning_rate': 0.21000000000000002} -47704.984833041126
MAPE: 0.3304757716730405
RMSE: 50280.708576254794


In [ ]:
model_xgb.gamma = 0
model_xgb.learning_rate = 0.21000000000000002

In [ ]:
params4 = {'subsample' : [0.5, 0.6, 0.8, 0.9, 1.0],'colsample_bytree' : [0.6, 0.8, 0.9, 1.0]}
grid_search4 = GridSearchCV(estimator=model_xgb, param_grid=params4, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
grid_search4.fit(x_train, y_train)
print(grid_search4.best_params_, grid_search4.best_score_)

y_pred = grid_search4.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'colsample_bytree': 0.6, 'subsample': 1.0} -47688.13479440086
MAPE: 0.33645884650137536
RMSE: 50141.62380037563


In [ ]:
model_xgb.colsample_bytree = 0.6
model_xgb.subsample = 1.0

In [ ]:
params5 = {'reg_alpha': [0, 0.001, 0.005, 0.01], 'reg_lambda': [0, 0.001, 0.005, 0.01]}
grid_search5 = GridSearchCV(estimator=model_xgb, param_grid=params5, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
grid_search5.fit(x_train, y_train)
print(grid_search5.best_params_, grid_search5.best_score_)

y_pred = grid_search5.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'reg_alpha': 0.005, 'reg_lambda': 0.005} -47604.08948892429
MAPE: 0.3312270453669055
RMSE: 50120.87713238308


In [ ]:
model_xgb.reg_alpha = 0.005
model_xgb.reg_lambda = 0.005

In [ ]:
params6 = {'n_estimators' : np.arange(100, 1000, 100)}
grid_search6 = GridSearchCV(estimator=model_xgb, param_grid=params6, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
import time
start_time = time.time()

grid_search6.fit(x_train, y_train)
print(grid_search6.best_params_, grid_search6.best_score_)

grid_time = time.time() - start_time
print(f'Grid search time: {grid_time} seconds')

y_pred = grid_search6.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

{'n_estimators': 100} -47674.713537075426
Grid search time: 58.3209593296051 seconds
MAPE: 0.3393226691360869
RMSE: 50168.05579915705


In [ ]:
model_xgb.n_estimators = 100

In [ ]:
start_time = time.time()

model_xgb.fit(x_train, y_train)

fit_time = time.time() - start_time

print(f'Fitting time: {fit_time} seconds')

Fitting time: 1.4623734951019287 seconds


In [ ]:
start_time = time.time()

print(f'MAPE: {mean_absolute_percentage_error(y_test, model_xgb.predict(x_test))}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, model_xgb.predict(x_test)))}')
predict_time = time.time() - start_time

print(f'Prediction time: {predict_time} seconds')

MAPE: 0.3889175448179766
RMSE: 44764.74500922061
Prediction time: 0.1383061408996582 seconds


По результатам подбора параметров по валидационной выборке и оценки качества на тестовой: MAPE около 40%, что конечно много, а RMSE около ~45000, что еще больше..Но, учитывая ошибки на предыдущей модели линейной регрессии, прослеживается большое улучшение.
Касаемо времени, после попытки всунуть в GridSearch все параметры, я думала, что он не загрузится никогда (спойлер: он не загрузился), поэтому я подбирада постепенно, что было довольно быстро, но в точных числах все еще почти минута (58 секунд). Время обучения и предсказания итоговой модели было достаточно быстрым (1,4 и 0,4 секунд).

## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
import time
from catboost import CatBoostRegressor

grid = {"learning_rate": [0.03, 0.1],
          "depth": [4, 6, 10],
          'l2_leaf_reg': [1, 3, 5, 7, 9],
          'iterations' : [10, 15, 30, 50]
}

cat = CatBoostRegressor()

start_time = time.time()

grid_search_result = cat.grid_search(
    grid, X=x_train, y=y_train, verbose=False, plot=True
)

grid_time = time.time() - start_time

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 148231.5108791	test: 147019.0152995	best: 147019.0152995 (0)	total: 1.26ms	remaining: 11.4ms
1:	learn: 144500.9789431	test: 143310.2471807	best: 143310.2471807 (1)	total: 2.38ms	remaining: 9.53ms
2:	learn: 140826.1975495	test: 139637.4185442	best: 139637.4185442 (2)	total: 3.42ms	remaining: 7.98ms
3:	learn: 137413.6486909	test: 136245.5919753	best: 136245.5919753 (3)	total: 4.43ms	remaining: 6.65ms
4:	learn: 134147.4306191	test: 132986.5568854	best: 132986.5568854 (4)	total: 5.11ms	remaining: 5.11ms
5:	learn: 130897.5717859	test: 129736.9436773	best: 129736.9436773 (5)	total: 6.13ms	remaining: 4.08ms
6:	learn: 127840.1872186	test: 126699.5198037	best: 126699.5198037 (6)	total: 7.12ms	remaining: 3.05ms
7:	learn: 124797.9253852	test: 123661.2383684	best: 123661.2383684 (7)	total: 8.24ms	remaining: 2.06ms
8:	learn: 121887.1912676	test: 120771.9808754	best: 120771.9808754 (8)	total: 9.29ms	remaining: 1.03ms
9:	learn: 119097.7498075	test: 118015.1242537	best: 118015.1242537 (9)	to

In [ ]:
print(f'Grid search time: {grid_time} seconds')

Grid search time: 11.533675193786621 seconds


In [ ]:
grid_search_result["params"]

{'depth': 10, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 50}

In [ ]:
cat.depth = 10
cat.learning_rate = 0.1
cat.l2_leaf_reg = 5
cat.iterations = 50

In [ ]:
y_pred = cat.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

MAPE: 0.3658288389858803
RMSE: 50188.38599354412


In [ ]:
start_time = time.time()

cat.fit(x_train, y_train)

fit_time = time.time() - start_time

0:	learn: 61269.7885082	total: 10.6ms	remaining: 519ms
1:	learn: 59588.9272660	total: 21ms	remaining: 505ms
2:	learn: 58070.8728310	total: 25.5ms	remaining: 399ms
3:	learn: 56896.5287530	total: 35.3ms	remaining: 406ms
4:	learn: 55768.7072516	total: 41.3ms	remaining: 371ms
5:	learn: 54916.2071568	total: 54.9ms	remaining: 402ms
6:	learn: 53932.7468449	total: 68.7ms	remaining: 422ms
7:	learn: 53168.5394976	total: 82.5ms	remaining: 433ms
8:	learn: 52493.1641761	total: 96.3ms	remaining: 439ms
9:	learn: 51954.6412906	total: 110ms	remaining: 441ms
10:	learn: 51411.6790638	total: 130ms	remaining: 462ms
11:	learn: 50906.2309199	total: 146ms	remaining: 462ms
12:	learn: 50503.9755006	total: 165ms	remaining: 469ms
13:	learn: 50207.5182360	total: 171ms	remaining: 441ms
14:	learn: 49902.5886365	total: 184ms	remaining: 430ms
15:	learn: 49627.4530422	total: 205ms	remaining: 436ms
16:	learn: 49303.5380142	total: 222ms	remaining: 430ms
17:	learn: 49069.0192121	total: 233ms	remaining: 415ms
18:	learn: 48

In [ ]:
print(f'Fitting time: {fit_time} seconds')

Fitting time: 0.7652831077575684 seconds


In [ ]:
start_time = time.time()

print(f'MAPE: {mean_absolute_percentage_error(y_test, cat.predict(x_test))}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, cat.predict(x_test)))}')

predict_time = time.time() - start_time
print(f'Predict time: {fit_time} seconds')

MAPE: 0.4528651187989599
RMSE: 46514.02071216461
Predict time: 0.7652831077575684 seconds


MAPE увеличилась по сравнению с XGBoost примерно на 5-7%, при этом RMSE увеличилась где-то 2000. В целом, это не совсем кординальная разница (и возможно я как то не докрутила кэтбустинг). Но вот касаемо скорости подбора гиперпараметров, кэтбустинг намного быстрее и продуктивнее перебирает параметры (11 секунд против почти минуты), а также в целом обучает(0.7 секунд), но вот предсказывает на 0,4 секунды медленее хах.

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [ ]:
from catboost import Pool

y = df.salary_in_usd
X = df.drop("salary_in_usd", axis=1)

In [ ]:
cat_features = [i for i, dtype in enumerate(X.dtypes) if str(dtype) == "object"]

print(cat_features)

[1, 2, 3, 4, 5, 7, 8]


In [ ]:
pool = Pool(data=X, label = y, cat_features=cat_features)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_temp, y_train, y_temp = train_test_split(X, y, random_state = 42, test_size = 0.2)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, random_state = 42, test_size = 0.5)

In [ ]:
grid = {"learning_rate": [0.03, 0.1],
          "depth": [4, 6, 10],
          'l2_leaf_reg': [1, 3, 5, 7, 9],
          'iterations' : [10, 15, 30, 50]
}

model = CatBoostRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=grid, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
start_time = time.time()

grid_search.fit(x_train, y_train, cat_features = cat_features)
print(grid_search.best_params_, grid_search6.best_score_)

grid_time = time.time() - start_time

Streaming output truncated to the last 5000 lines.
1:	learn: 62139.4289681	total: 6.59ms	remaining: 26.3ms
2:	learn: 61574.1546335	total: 8.22ms	remaining: 19.2ms
3:	learn: 61131.7501216	total: 12.9ms	remaining: 19.3ms
4:	learn: 60664.4211430	total: 15.2ms	remaining: 15.2ms
5:	learn: 60186.8872613	total: 17.5ms	remaining: 11.7ms
6:	learn: 59739.0690118	total: 20.5ms	remaining: 8.78ms
7:	learn: 59313.7119587	total: 21.6ms	remaining: 5.41ms
8:	learn: 58915.1283722	total: 23.4ms	remaining: 2.6ms
9:	learn: 58503.0200321	total: 25.2ms	remaining: 0us
0:	learn: 62712.5767489	total: 1.99ms	remaining: 17.9ms
1:	learn: 62248.7502555	total: 8.74ms	remaining: 35ms
2:	learn: 61787.0325581	total: 13.1ms	remaining: 30.5ms
3:	learn: 61369.2501736	total: 21.3ms	remaining: 31.9ms
4:	learn: 60916.5421543	total: 24ms	remaining: 24ms
5:	learn: 60484.5053111	total: 27.4ms	remaining: 18.3ms
6:	learn: 60075.0413719	total: 29.2ms	remaining: 12.5ms
7:	learn: 59730.2534241	total: 35.9ms	remaining: 8.98ms
8:	lear

In [ ]:
print(f'Grid search time: {grid_time} seconds')

Grid search time: 80.7724118232727 seconds


In [ ]:
y_pred = grid_search.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

MAPE: 0.37160458067473495
RMSE: 50086.73777001365


In [ ]:
model = CatBoostRegressor()
model.depth = 6
model.learning_rate = 0.1
model.l2_leaf_reg = 5
model.iterations = 50

In [ ]:
start_time = time.time()

model.fit(
    x_train,
    y_train,
    cat_features=cat_features,
    eval_set=(x_val, y_val),
    verbose=False,
)

fit_time = time.time() - start_time
print(f'Fitting time: {fit_time} seconds')

Fitting time: 6.33806586265564 seconds


In [ ]:
start_time = time.time()

print(f'MAPE: {mean_absolute_percentage_error(y_test, model.predict(x_test))}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, model.predict(x_test)))}')

predict_time = time.time() - start_time
print(f'Predict time: {fit_time} seconds')

MAPE: 0.41394340763601023
RMSE: 46032.48343365925
Predict time: 6.33806586265564 seconds


**Ответ:** Так-так....ну что-то неутешительные результаты в плане времени выполнения. Это скорее всего связано с тем, что пришлось использовать обычный Grid Search из sklearn, нежели чем, как при первой попытке кэтбустинга c помощью catboost.grid_search, из-за чего все получилось гораздо и гораздо быстрее, чем сейчас...в первый раз занчло минуту, а во второй 80 секунд..в общем не получилось быстрее, конкретно в данной ситуации, когда использовали pool для смешанных данных. Все таки кодировка показала себя лучше (либо я глупая)
А вот с точки зрения ошибки, MAPE чуть упал, около 4%, RMSE остался примерно на таком же уровне, изменение вообще из-за того, что в этом случае grod search выбрад другие лучшие параметры

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import time
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [ ]:
from lightgbm import LGBMRegressor


params = {
    "max_depth" : [3, 4, 5, 10],
    "learning_rate" : [0.01, 0.1, 1],
    "n_estimators" : [20, 40]

}

estimator = lgb.LGBMRegressor()


start_time = time.time()

gbm = GridSearchCV(estimator, params, cv=3)
gbm.fit(x_train, y_train)

gridt_time = time.time() - start_time

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 2002, number of used features: 36
[LightGBM] [Info] Start training from score 137264.451548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
print(f'Grid time: {gridt_time} seconds')
print(gbm.best_params_)

Grid time: 5.259232997894287 seconds
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 40}


In [ ]:
estimator.learning_rate = 0.1
estimator.max_depth = 4
estimator.n_estimators = 40

In [ ]:
params2 = { "num_leaves": [31, 25, 50, 70],
    "reg_alpha": [0.0, 0.1, 0.5],
    "reg_lambda": [0.0, 0.1, 0.5] }

start_time = time.time()

gbm = GridSearchCV(estimator, params2, cv=3)
gbm.fit(x_train, y_train)

grid_time = time.time() - start_time

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000452 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 2002, number of used features: 36
[LightGBM] [Info] Start training from score 137264.451548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
print(f'Grid time: {grid_time} seconds')
print(gbm.best_params_)

Grid time: 6.279707908630371 seconds
{'num_leaves': 31, 'reg_alpha': 0.0, 'reg_lambda': 0.5}


In [ ]:
estimator.num_leaves = 31
estimator.reg_alpha = 0.0
estimator.reg_lambda = 0.5

In [ ]:
y_pred = gbm.predict(x_val)

print(f'MAPE: {mean_absolute_percentage_error(y_val, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}')

MAPE: 0.33830102650539706
RMSE: 49868.22979760292


In [ ]:
estimator.fit(x_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 82
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 39
[LightGBM] [Info] Start training from score 138055.989348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

LGBMRegressor(max_depth=4, n_estimators=40, reg_lambda=0.5)

In [ ]:
start_time = time.time()

print(f'MAPE: {mean_absolute_percentage_error(y_test, estimator.predict(x_test))}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, estimator.predict(x_test)))}')

predict_time = time.time() - start_time
print(f'Predict time: {predict_time} seconds')

MAPE: 0.41103170501020153
RMSE: 46334.10046853716
Predict time: 0.049524784088134766 seconds


Опять же не удалось сделать грид черч одним махом, пришлось разбирать на подбор отдельных параметров, что по отдельности заняло где-то по пять секунд, а все вместе не загрузилось бы и до вечера, наверно..Касаемо ошибок -- они снова почти такие же как на предыдущих моделях, то есть достаточно большие.

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:** Понятное дело, что худший результат по качеству показала модель линейной регрессии, из-за того что она не подходит для такого типа данных.

Если говорить конкретно о моделях бустинга, то лучшим в плане ошибки оказался XGBoost с MAPE около 39% и RMSE около 45 000, остальные модели имеют примерно похожие ошибки, которые чуть больше, чем у лучшей модели (особенно MAPE). По времени худшим оказался CatBoosting, но нужно брать во внимание, что я сделала грид серч сразу по всем параметрам и это все заняло 80 секунд, при этом в XGBoost я разбивала параметры на 6 частей и отдельно делала грид серо, каждый из которых занял около минуты. Поэтому справедливо будет сказать, что худшим по времени будет именно XGBoost. С другой стороны, лучшей по времени получился обычный catboost, потому что он с помощью внутреннего аттрибута сделала грид серч по всем параметрам за 11 секунд, также можно отметить LightGBM, который при разбитии на отдельные группы для грид серча, выполнен каждый где то за 6 секунд.

Обычное обучение и предсказание моделей не отличалось каким то большим отрывом по времени, все проходили довольно быстро.


# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
ben harper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --